In [ ]:
import numpy as np

import matplotlib.pyplot as pl
import matplotlib as mpl

mpl.rc('figure', figsize=(14,12))

import os
import re

from scipy.io import loadmat
from scipy.ndimage import gaussian_filter1d

In [ ]:
data_folder='../../Data/Data-10-09/matFiles'

listFiles = os.listdir(data_folder)

listFilesMat=[]

for filename in listFiles:
    if re.match('.*.mat', filename):
        listFilesMat.append(filename)
listFilesMat.sort()

In [ ]:
listFilesMat

### Functions

In [ ]:
#filter ind if necessary
begin_ind=175   #27
end_ind=912  #49

In [ ]:
def split_name(filename):
    '''returns (picNumber, masker filename)'''
    m=re.match('p([0-9]{4})_fmasked_CAP_(.*).mat', filename)
    assert m, f'{filename} dit not match'
    p, name= m.groups()
    return int(p), name

map_table={}  #map masker filename -> pic numbers
for fln in listFilesMat:
    p, name = split_name(fln)
    if name not in map_table:
        map_table[name]=[p]
    else:
        li=map_table[name]
        li.append(p)
        map_table[name]=li
        
filtered_map_table={}  #map masker filename -> pic numbers
for fln in listFilesMat:
    p, name = split_name(fln)
    if p>=begin_ind and p<=end_ind:

        if name not in filtered_map_table:
            filtered_map_table[name]=[p]
        else:
            li=filtered_map_table[name]
            li.append(p)
            filtered_map_table[name]=li
    
    

In [ ]:
filtered_map_table

As we messed up the things with attn5dB, needs to check if attenuations are the same

In [ ]:
def loadPic(n):
    found=False
    for filename in listFilesMat:
        if f'p{str(n).zfill(4)}' in filename:
            found=True
            break
    assert found, f'pic {n} not found.'
    arr= loadmat(f'{data_folder}/{filename}')
    return arr


def plotPic(picDic=None, n=None, label='', tmin=0, baseline=None):
    '''
    Plot pic given picture dictionary or pic number. 
    Args:
        baseline: Picture dictionary for baseline signal'''
    if not n is None:
        picDic=loadPic(n)
    assert not picDic is None
    if label=='':
        label=picDic['wavefilename'][0]
    val=np.squeeze(picDic['valAvg'].T)
    t=np.linspace(picDic['XstartPlot_ms'][0][0], picDic['XendPlot_ms'][0][0], num=len(val))
    if baseline is not None:
        val=val-np.squeeze(baseline['valAvg'].T)
    #val=np.cumsum(val) #HACK!!
    pl.plot(t, val, label=label)
    pl.xlim([tmin, picDic['XendPlot_ms'][0][0]])
    
def getMaskerSig(dic, name):
    
    picNums=dic[name]
    firstPic=True
    for picNum in picNums:
        picDic=loadPic(picNum)
        if firstPic:
            val=np.sum(picDic['valAll'][1::2], axis=0)
            #val=np.squeeze(picDic['valAvg'].T)
            firstPic=False
        else:
            val+=np.sum(picDic['valAll'][1::2], axis=0)
            #val+=np.squeeze(picDic['valAvg'].T)
    val/=len(picNums)
    return val
    
def plotMasker(dic, name, tmin=6, tmax=12, baseline=None):
    '''
    Plot avg signal for masker given masker name and dictionary of picNums 
    Args:
        baseline: signal for baseline signal'''
    label=name
    val=getMaskerSig(dic, name)
    
    picNums=dic[name]
    for picNum in picNums:
        picDic=loadPic(picNum)
        break

    t=np.linspace(picDic['XstartPlot_ms'][0][0], picDic['XendPlot_ms'][0][0], num=len(val))
    if baseline is not None:
        val=val-baseline
    #val=gaussian_filter1d(val, 1.5) #HACK
    #val=np.cumsum(val) #HACK!!
    pl.plot(t, val, label=label)
    if tmax==0:
        tmax=picDic['XendPlot_ms'][0][0]
    pl.xlim([tmin, tmax])
    
    
def getMaskerAtten(picDic=None, n=None):
    if not n is None:
        picDic=loadPic(n)
    assert not picDic is None
    ans=picDic['masker_atten_dB']
    return ans[0][0]

In [ ]:
filtered_map_table2

In [ ]:
filtered_map_table2={}
for fln, picNums in filtered_map_table.items():
    temp={}
    max_amp=0
    no_band=False
    for n in picNums:
        p=loadPic(n)
        if len(p['bands'])>0:
            amp=p['bands'][0][0][0][0][0]
            if amp>max_amp:
                max_amp=amp
            if not amp in temp:
                temp[amp]= [n]
            else:
                temp[amp].append(n)
        else:
            no_band=True
            break
    if no_band:
        filtered_map_table2[fln]=picNums
    else:
        for amp in temp:
            attn=amp/max_amp
            
            if np.abs(attn-1.)<1e-4:
                filtered_map_table2[fln]=temp[amp]
            elif np.abs(attn-0.1)<1e-4:
                filtered_map_table2[fln+'_attn20dB']=temp[amp]  #et mince....
            elif  np.abs(attn-10**(-1/4))<1e-4:
                filtered_map_table2[fln+'_attn5dB']=temp[amp]
            elif np.abs(attn-10**(-3/4))<1e-4:
                filtered_map_table2[fln+'_attn15dB']=temp[amp]
            else:
                print(attn)
                

### Show examples of signals

In [ ]:
dic=filtered_map_table2

In [ ]:
p=loadPic(205)
print(np.shape(p['valAll']))

In [ ]:
names=list(dic.keys())
names.sort()
print(names)

In [ ]:
broadband=getMaskerSig(dic, 'broadband_noise')
nomasker=getMaskerSig(dic, 'nomasker')

In [ ]:
plotMasker(dic, 'nomasker')
plotMasker(dic, 'broadband_noise')
pl.legend()

### Narrow-band analysis

In [ ]:
plotMasker(dic, '1_hp_10000Hz')
plotMasker(dic, '2_hp_9000Hz')
plotMasker(dic, '3_hp_7500Hz')
plotMasker(dic, '4_hp_6000Hz')
plotMasker(dic, '5_hp_5000Hz')
plotMasker(dic, '6_hp_4000Hz')
plotMasker(dic, '7_hp_3000Hz')
plotMasker(dic, '8_hp_2000Hz')
plotMasker(dic, '9_hp_1500Hz')
plotMasker(dic, '10_hp_1000Hz')
plotMasker(dic, 'broadband_noise')  #5 dB higher
pl.legend()

In [ ]:
#plotMasker(dic, '1_hp_10000Hz', baseline=nomasker)
plotMasker(dic, '2_hp_9000Hz', baseline=getMaskerSig(dic, '1_hp_10000Hz'))
plotMasker(dic, '3_hp_7500Hz', baseline=getMaskerSig(dic, '2_hp_9000Hz'))
plotMasker(dic, '4_hp_6000Hz', baseline=getMaskerSig(dic, '3_hp_7500Hz'))
plotMasker(dic, '5_hp_5000Hz', baseline=getMaskerSig(dic, '4_hp_6000Hz'))
plotMasker(dic, '6_hp_4000Hz', baseline=getMaskerSig(dic, '5_hp_5000Hz'))
plotMasker(dic, '7_hp_3000Hz', baseline=getMaskerSig(dic, '6_hp_4000Hz'))
plotMasker(dic, '8_hp_2000Hz', baseline=getMaskerSig(dic, '7_hp_3000Hz'))
plotMasker(dic, '9_hp_1500Hz', baseline=getMaskerSig(dic, '8_hp_2000Hz'))
plotMasker(dic, '10_hp_1000Hz', baseline=getMaskerSig(dic, '9_hp_1500Hz'))
pl.legend()

In [ ]:
plotMasker(dic, '1_hp_10000Hz', baseline=broadband)
plotMasker(dic, '2_hp_9000Hz', baseline=broadband)
plotMasker(dic, '3_hp_7500Hz', baseline=broadband)
plotMasker(dic, '4_hp_6000Hz', baseline=broadband)
plotMasker(dic, '5_hp_5000Hz', baseline=broadband)
plotMasker(dic, '6_hp_4000Hz', baseline=broadband)
plotMasker(dic, '7_hp_3000Hz', baseline=broadband)
plotMasker(dic, '8_hp_2000Hz', baseline=broadband)
plotMasker(dic, '9_hp_1500Hz', baseline=broadband)
plotMasker(dic, '10_hp_1000Hz', baseline=broadband)
pl.legend()

In [ ]:
plotMasker(dic, '1_hp_10000Hz', baseline=nomasker)
plotMasker(dic, '2_hp_9000Hz', baseline=nomasker)
plotMasker(dic, '3_hp_7500Hz', baseline=nomasker)
plotMasker(dic, '4_hp_6000Hz', baseline=nomasker)
plotMasker(dic, '5_hp_5000Hz', baseline=nomasker)
plotMasker(dic, '6_hp_4000Hz', baseline=nomasker)
plotMasker(dic, '7_hp_3000Hz', baseline=nomasker)
plotMasker(dic, '8_hp_2000Hz', baseline=nomasker)
plotMasker(dic, '9_hp_1500Hz', baseline=nomasker)
plotMasker(dic, '10_hp_1000Hz', baseline=nomasker)
pl.legend()

### Notch 5k

In [ ]:
plotMasker(dic, '1_notch5k_3k_attn65dB')
plotMasker(dic, '2_notch5k_3k_attn60dB')
plotMasker(dic, '3_notch5k_3k_attn55dB')
plotMasker(dic, '4_notch5k_3k_attn50dB')
plotMasker(dic, '5_notch5k_3k_attn45dB')
plotMasker(dic, '6_notch5k_3k_attn40dB')
plotMasker(dic, '7_notch5k_3k_attn37dB')
plotMasker(dic, '8_notch5k_3k_attn34dB')
plotMasker(dic, '9_notch5k_3k_attn31dB')
plotMasker(dic, '10_notch5k_3k_attn28dB')
plotMasker(dic, '11_notch5k_3k_attn25dB')
plotMasker(dic, '12_notch5k_3k_attn22dB')

plotMasker(dic, 'broadband_noise')
pl.legend()

In [ ]:
#plotMasker(dic, '1_notch5k_3k_attn65dB', baseline=broadband)
#plotMasker(dic, '2_notch5k_3k_attn60dB', baseline=broadband)
plotMasker(dic, '3_notch5k_3k_attn55dB', baseline=broadband)
plotMasker(dic, '4_notch5k_3k_attn50dB', baseline=broadband)
plotMasker(dic, '5_notch5k_3k_attn45dB', baseline=broadband)
plotMasker(dic, '6_notch5k_3k_attn40dB', baseline=broadband)
plotMasker(dic, '7_notch5k_3k_attn37dB', baseline=broadband)
plotMasker(dic, '8_notch5k_3k_attn34dB', baseline=broadband)
plotMasker(dic, '9_notch5k_3k_attn31dB', baseline=broadband)
plotMasker(dic, '10_notch5k_3k_attn28dB', baseline=broadband)
plotMasker(dic, '11_notch5k_3k_attn25dB', baseline=broadband)
plotMasker(dic, '12_notch5k_3k_attn22dB', baseline=broadband)

pl.legend()

In [ ]:
cap=[]
rms=[]
for masker in ['3_notch5k_3k_attn55dB','4_notch5k_3k_attn50dB',
              '5_notch5k_3k_attn45dB','6_notch5k_3k_attn40dB',
              '7_notch5k_3k_attn37dB','8_notch5k_3k_attn34dB',
              '9_notch5k_3k_attn31dB','10_notch5k_3k_attn28dB',
              '11_notch5k_3k_attn25dB', '12_notch5k_3k_attn22dB']:
    sig=getMaskerSig(dic,masker)
    cap.append(np.max(sig-broadband)-np.min(sig-broadband))
    rms.append(np.std(sig-broadband))
pl.plot(-np.array([55,50,45,40,37,34,31,28,25,22]), cap, label='max-min')

#pl.plot(-np.array([55,50,45,40,37,34,31,28,25,22]), np.array(rms)*10, label='rms x10')
pl.legend()
    

### Notch 2k

In [ ]:
plotMasker(dic, '2_notch2500_2k_attn65dB')
plotMasker(dic, '3_notch2500_2k_attn60dB')
plotMasker(dic, '4_notch2500_2k_attn55dB')
plotMasker(dic, '5_notch2500_2k_attn45dB')
plotMasker(dic, '6_notch2500_2k_attn40dB')
plotMasker(dic, '7_notch2500_2k_attn37dB')
plotMasker(dic, '8_notch2500_2k_attn34dB')
plotMasker(dic, '9_notch2500_2k_attn31dB')
plotMasker(dic, '10_notch2500_2k_attn28dB')
plotMasker(dic, '11_notch2500_2k_attn25dB')
plotMasker(dic, '12_notch2500_2k_attn22dB')

plotMasker(dic, 'broadband_noise')
pl.legend()

In [ ]:
#plotMasker(dic, '2_notch2500_2k_attn65dB', baseline=broadband)
plotMasker(dic, '3_notch2500_2k_attn60dB', baseline=broadband)
plotMasker(dic, '4_notch2500_2k_attn55dB', baseline=broadband)
plotMasker(dic, '5_notch2500_2k_attn45dB', baseline=broadband)
plotMasker(dic, '6_notch2500_2k_attn40dB', baseline=broadband)
plotMasker(dic, '7_notch2500_2k_attn37dB', baseline=broadband)
plotMasker(dic, '8_notch2500_2k_attn34dB', baseline=broadband)
plotMasker(dic, '9_notch2500_2k_attn31dB', baseline=broadband)
plotMasker(dic, '10_notch2500_2k_attn28dB', baseline=broadband)
plotMasker(dic, '11_notch2500_2k_attn25dB', baseline=broadband)
plotMasker(dic, '12_notch2500_2k_attn22dB', baseline=broadband)

#plotMasker(dic, 'broadband_noise')
pl.legend()

In [ ]:
s=getMaskerSig(dic,'12_notch2500_2k_attn22dB')+getMaskerSig(dic,'11_notch2500_2k_attn25dB')
s2=getMaskerSig(dic,'3_notch2500_2k_attn60dB')+getMaskerSig(dic,'4_notch2500_2k_attn55dB')
baseline=getMaskerSig(dic,'broadband_noise')
t=np.arange(len(s))/(48828)
pl.plot(t*1e3, s-2*baseline, label='low atten')
pl.plot(t*1e3, s2-2*baseline, label='high atten')
pl.xlim([6,12])
pl.legend()

### lp

In [ ]:
plotMasker(dic, '4_lp_2500Hz')
plotMasker(dic, '6_lp_4000Hz')
plotMasker(dic, '7_lp_5000Hz')
plotMasker(dic, '10_lp_7000Hz')

In [ ]:
baseline=getMaskerSig(dic,'10_hp_1000Hz')
plotMasker(dic, '4_lp_2500Hz', baseline=baseline)
plotMasker(dic, '6_lp_4000Hz',  baseline=baseline)
plotMasker(dic, '7_lp_5000Hz',  baseline=baseline)
plotMasker(dic, '10_lp_7000Hz',  baseline=baseline)
pl.legend()

In [ ]:
plotMasker(dic, '4_lp_2500Hz', baseline=nomasker)
plotMasker(dic, '6_lp_4000Hz',  baseline=nomasker)
plotMasker(dic, '7_lp_5000Hz',  baseline=nomasker)
plotMasker(dic, '10_lp_7000Hz',  baseline=nomasker)
pl.legend()

### Click delay

In [ ]:
clickdelay_map_table={'8ms':[923,924], '10ms':[913,914], '14ms': [917,918], '12ms': [919,920]}  

In [ ]:
for delay in clickdelay_map_table:
    plotMasker(clickdelay_map_table, delay)
plotMasker(dic, 'nomasker')
pl.legend()

### Narrow 5k

In [ ]:
#plotMasker(dic, '1_narrow5k_2k')  #-20db add. attenuation...
plotMasker(dic, '1_narrow5k_bw1700_attn20dB')
plotMasker(dic, '2_narrow5k_bw1400_attn20dB')
plotMasker(dic, '3_narrow5k_bw1100_attn20dB')
pl.legend()

In [ ]:
#plotMasker(dic, '1_narrow5k_2k', baseline=nomasker) #-20db add. attenuation...
plotMasker(dic, '1_narrow5k_bw1700_attn20dB', baseline=nomasker)
plotMasker(dic, '2_narrow5k_bw1400_attn20dB', baseline=nomasker)
plotMasker(dic, '3_narrow5k_bw1100_attn20dB', baseline=nomasker)
plotMasker(dic, '4_narrow5k_bw900_attn20dB', baseline=nomasker)
pl.legend()

### Notches 5k

Too much attenuation

### Narrow around 5k

In [ ]:

plotMasker(dic, '4_hp_6000Hz')
plotMasker(dic, '5_hp_5000Hz')
plotMasker(dic, '6_hp_4000Hz')

plotMasker(dic, '1_hp_5500Hz')
plotMasker(dic, '5_hp_4500Hz')
pl.legend()

In [ ]:
plotMasker(dic, '4_hp_6000Hz', baseline=broadband)

plotMasker(dic, '1_hp_5500Hz', baseline=broadband)
plotMasker(dic, '5_hp_5000Hz', baseline=broadband)

plotMasker(dic, '5_hp_4500Hz', baseline=broadband)
plotMasker(dic, '6_hp_4000Hz', baseline=broadband)

pl.legend()